In [1]:
from dotenv import load_dotenv
import os
load_dotenv(dotenv_path='.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_n


문제 1-1 : 기본 Chain 만들기 - AI 요리사

In [4]:
import langchain
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template("You're a creative and smart chef. \
    답변은 반드시 '추천 요리', '재료', '조리법'으로 구성해 주세요. 그리고 '조리법'은 단계별로 번호를 붙여 설명해 주세요. <재료>: {input} \
    Answer the question. <Question>: {input}로 만들 수 있는 요리를 추천해 주세요.")

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

result = chain.invoke({"input": "계란, 밥, 김치"})
print(result)

### 추천 요리 

*   **김치볶음밥**

### 재료 

*   밥 2컵 
*   김치 100g 
*   계란 1개 
*   참기름 1숟가락 
*   간장 1숟가락 

### 조리법 

1.  김치를 잘게 썰어주세요.
2.  팬에 참기름을 두르고 썰어 놓은 김치를 볶아주세요.
3.  김치가 적당히 볶아지면 밥을 넣고 함께 볶아주세요.
4.  간장으로 간을 맞춰주세요.
5.  다른 팬에 계란을 풀어 스크램블을 만들어주세요.
6.  볶음밥에 스크램블을 올리고 잘 섞어주세요.


문제 1-2 : 2단계 Multi Chain 만들기 - 여행지 정보 시스템

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

from pprint import pprint

prompt1 = ChatPromptTemplate.from_messages([
    ("system", "특정 지역의 대표적인 관광 명소 한 곳을 추천해 주세요. 다른 설명 없이 명소의 이름만 정확하게 답변해야 합니다."),
    ("user", "{location}의 대표적인 관광 명소는 어디인가요?")
])

prompt2 = ChatPromptTemplate.from_messages([
    ("system",
     """당신은 특정 관광 명소의 역사, 특징, 방문 팁을 상세하게 설명해 주는 여행 가이드입니다.
     아래 형식에 맞춰서 부드러운 서술형 문장으로 작성해주세요.
    
     명소: [명소 이름]

     역사:
     [여기에 역사에 대한 설명을 2-3문장의 서술형으로 작성]
     
     특징:
     [여기에 특징에 대한 설명을 2-3문장의 서술형으로 작성]

     방문 팁:
     [여기에 방문 팁을 2-3문장의 서술형으로 작성]"""),
    ("user", "{landmark}에 대한 상세한 정보를 알려주세요.")
])

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

chain1 = prompt1 | llm | StrOutputParser()

chain2 = (
    {"landmark": chain1}
    | prompt2
    | llm
    | StrOutputParser()
)

response = chain2.invoke({"location": "로마"})

pprint(response)

('명소: 콜로세움\n'
 '\n'
 '역사:\n'
 '로마 제국의 상징인 콜로세움은 서기 72년 베스파시아누스 황제에 의해 착공되어 80년 티투스 황제 시대에 완성된 원형 경기장입니다. 약 '
 '5만 명의 관중을 수용할 수 있었던 이 거대한 건축물은 검투사들의 결투와 야수 사냥, 그리고 해상 전투를 재현하는 해전 연출 등이 '
 '펼쳐지던 장소로 400여 년간 로마인들의 열광적인 사랑을 받았습니다.\n'
 '\n'
 '특징:\n'
 '외부에서 보면 4층 규모의 아치 구조물이 겹겹이 쌓인 웅장한 모습을 자랑하는 콜로세움은 내부가 완전 개방된 공간으로 설계되어 있어, '
 "지금도 2천 년이 지난 지금까지도 그 규모감에 압도됩니다. 특히 무대 아래 지하 공간인 '히포지움'에서는 검투사들과 야수들이 등장하는 "
 '기계 장치와 함정 문 시설이 남아 있어 당시의 기술력을 실감하게 합니다.\n'
 '\n'
 '방문 팁:\n'
 '콜로세움은 오전 일찍 또는 오후 늦게 방문하면 붐비는 인파를 피할 수 있으며, 특히 황금빛 석양이 내부에 비추는 저녁 시간대가 사진 '
 '촬영에 최적입니다. 온라인으로 로마 포럼과 패키지 티켓을 미리 구매하면 긴 줄을 피할 수 있고, 무료 입장일인 매월 첫 일요일을 노리면 '
 '입장료도 절약할 수 있으니 일정을 잘 확인하시기 바랍니다.')


문제 1-3 : FewShotPromptTemplate과 시스템 메시지 활용

In [15]:
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate, 
    ChatPromptTemplate
)
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
        "news": "현대자동차가 2025년형 아이오닉 7을 공개하며 전기차 시장 공략을 가속화한다. 이 모델은 한 번 충전으로 600km를 주행할 수 있으며, 친환경 소재를 대거 사용하여 탄소 배출량 감소에 기여할 것으로 기대된다.",
        "keywords": "현대자동차, 아이오닉 7, 전기차"
    }
]

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "뉴스 키워드 추출 전문가입니다. 핵심 키워드 3개를 추출하세요."),
    few_shot_prompt,
    ("human", "{input}")
])

chain = final_prompt | llm | StrOutputParser()

test_news = "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다.\n모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다.\n내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."

result = chain.invoke({"input": test_news})

print("--- 테스트 뉴스 ---")
print(test_news)
print("\n--- 예상 출력 형식 ---")
print(result)

--- 테스트 뉴스 ---
제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다.
모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다.
내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다.

--- 예상 출력 형식 ---
키워드: 제미나이, AI, 구글
